In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from time import time
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
with open("train.json") as f:
    datastore = json.loads(f.read())
f.close()
with open("test.json") as f:
    test = json.loads(f.read())
f.close()
print(type(datastore[0]))
print(datastore[0])
print(datastore[0]['ingredients'])
print(len(datastore[0]['ingredients']))
length = len(datastore)
length1 = len(test)
max = 0
pos = 0
for x in range(length):
    if(len(datastore[x]['ingredients']) > max):
        max = len(datastore[x]['ingredients'])
        pos = x
print(max)
print(datastore[pos]['id'])
print(length)

<class 'dict'>
{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}
['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']
9
65
3885
39774


In [3]:
cuisine = []
ingre = []
full = []
for i in range(length):
    cuisine.append(datastore[i]['cuisine'])
    for j in range(len(datastore[i]['ingredients'])):
        ingre.append(datastore[i]['ingredients'][j])
    li = [y for y in ingre]
    li = ",".join(map(str, li))
    full.append(li)
    ingre = []
    #if(i == 5000):
    #    break
    if(i%5000 == 0):
        print(i)
df = pd.DataFrame({'cuisine':cuisine,'ingredients':full})
#print(df.head())
#df.groupby('cuisine').cuisine.count().plot.bar(ylim=0)
#plt.show()



0
5000
10000
15000
20000
25000
30000
35000


In [4]:
print(df.head())
df1 = df.copy()
cuis = np.unique(df1['cuisine'].values)
print(cuis)
print(cuis[0])
b = {cuis[i]:i for i in range(len(cuis))}
print(b)
df1 = df1.replace({'cuisine':b})
print(df1.head())

       cuisine                                        ingredients
0        greek  romaine lettuce,black olives,grape tomatoes,ga...
1  southern_us  plain flour,ground pepper,salt,tomatoes,ground...
2     filipino  eggs,pepper,salt,mayonaise,cooking oil,green c...
3       indian                     water,vegetable oil,wheat,salt
4       indian  black pepper,shallots,cornflour,cayenne pepper...
['brazilian' 'british' 'cajun_creole' 'chinese' 'filipino' 'french'
 'greek' 'indian' 'irish' 'italian' 'jamaican' 'japanese' 'korean'
 'mexican' 'moroccan' 'russian' 'southern_us' 'spanish' 'thai'
 'vietnamese']
brazilian
{'brazilian': 0, 'british': 1, 'cajun_creole': 2, 'chinese': 3, 'filipino': 4, 'french': 5, 'greek': 6, 'indian': 7, 'irish': 8, 'italian': 9, 'jamaican': 10, 'japanese': 11, 'korean': 12, 'mexican': 13, 'moroccan': 14, 'russian': 15, 'southern_us': 16, 'spanish': 17, 'thai': 18, 'vietnamese': 19}
   cuisine                                        ingredients
0        6  romaine 

In [5]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,norm='l2',\
                       encoding='latin-1',ngram_range=(1,2))
print(df1['ingredients'][0])
features = tfidf.fit_transform(df1.ingredients).toarray()
labels = df1['cuisine']
print(features.shape)


romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles
(39774, 17927)


In [6]:
'''N = 2
for i in range(len(cuis)):
    features_chi2 = chi2(features,labels == i)
    indices = np.argsort(features_chi2[0])
    features_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in features_names if len(v.split(',')) == 1]
    bigrams = [v for v in features_names if len(v.split(',')) == 2]
    print("# '{}'".format(cuis[i]))
    print("  . Most correlated unigrams:\n. {}".\
          format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".\
          format('\n. '.join(bigrams[-N:])))
    '''

'N = 2\nfor i in range(len(cuis)):\n    features_chi2 = chi2(features,labels == i)\n    indices = np.argsort(features_chi2[0])\n    features_names = np.array(tfidf.get_feature_names())[indices]\n    unigrams = [v for v in features_names if len(v.split(\',\')) == 1]\n    bigrams = [v for v in features_names if len(v.split(\',\')) == 2]\n    print("# \'{}\'".format(cuis[i]))\n    print("  . Most correlated unigrams:\n. {}".          format(\'\n. \'.join(unigrams[-N:])))\n    print("  . Most correlated bigrams:\n. {}".          format(\'\n. \'.join(bigrams[-N:])))\n    '

In [10]:
X_train, X_test, y_train, y_test =\
train_test_split(df1['ingredients'],\
                 df1['cuisine'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
lgr = LogisticRegression(random_state=0).fit(X_train_tfidf, y_train)
lsvc = LinearSVC().fit(X_train_tfidf,y_train)



In [8]:
str1 = datastore[1]['ingredients']
print(str1)
str1 = ",".join(map(str, str1))
print(str1)
print( cuis[(clf.predict(count_vect.transform([str1])))] )

['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']
plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,milk,vegetable oil
['southern_us']


In [9]:
CV = 1
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

for model in models:
    accuracy = cross_val_score(model,features,labels,\
                            scoring='accuracy',cv=2)
    print(str(model),":",accuracy)
    print("\n\n")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False) : [0.32921128 0.32649263]



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0) : [0.77771075 0.77023289]



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) : [0.60267431 0.60696142]



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          ve

In [36]:
id1 = []
ingre1 = []
full1 = []
for i in range(length1):
    id1.append(test[i]['id'])
    for j in range(len
                   (test[i]['ingredients'])):
        ingre1.append(test[i]['ingredients'][j])
    li1 = [y for y in ingre1]
    li1 = ",".join(map(str, li1))
    full1.append(li1)
    ingre1 = []
    #if(i == 5000):
    #    break
    if(i%5000 == 0):
        print(i)
        
full2 = []
for k in range(len(full1)):
    full2.append(full1[k])

df2 = pd.DataFrame({'id':id1,'ingredients':full1})
print(df2.head())
print(df2['ingredients'][0])
ans = []

print("\n\n")
print(full2[:2])
print("\n\n\n")

for k in range(len(full1)):
    ans.append((cuis[(\
    clf.predict(count_vect.transform([full1[k]])))]))

out = pd.DataFrame({'id':df2['id'], 'cuisine':ans})
'''df2['ingredients'] = ",".join(map(str, df2['ingredients']))
print(df2.head())
ans = pd.DataFrame(columns=['cuisine'])
k = 0
print(df2['ingredients'][0])
print("\t\t\tTesting")'''
'''for i in df2['ingredients']:
    temp = pd.DataFrame(columns=['cuisine'])
    temp['cuisine'] = (cuis[(\
    clf.predict(count_vect.transform([i])))])
    #print(i)
    print(temp)
    ans = ans.append(temp)
    k+=10
    if(k == 1):
        break'''
#print(ans)
print(out.head())
out['cuisine'] = out['cuisine'].str[0]
print(out.head())

0
5000
      id                                        ingredients
0  18009  baking powder,eggs,all-purpose flour,raisins,m...
1  28583  sugar,egg yolks,corn starch,cream of tartar,ba...
2  41580  sausage links,fennel bulb,fronds,olive oil,cub...
3  29752  meat cuts,file powder,smoked sausage,okra,shri...
4  35687  ground black pepper,salt,sausage casings,leeks...
baking powder,eggs,all-purpose flour,raisins,milk,white sugar



['baking powder,eggs,all-purpose flour,raisins,milk,white sugar', 'sugar,egg yolks,corn starch,cream of tartar,bananas,vanilla wafers,milk,vanilla extract,toasted pecans,egg whites,light rum']




          cuisine     id
0   [southern_us]  18009
1   [southern_us]  28583
2       [italian]  41580
3  [cajun_creole]  29752
4       [italian]  35687
        cuisine     id
0   southern_us  18009
1   southern_us  28583
2       italian  41580
3  cajun_creole  29752
4       italian  35687


In [37]:
file = "submission.csv"
out.to_csv(file,index=False)

In [41]:
id1 = []
ingre1 = []
full1 = []
for i in range(length1):
    id1.append(test[i]['id'])
    for j in range(len
                   (test[i]['ingredients'])):
        ingre1.append(test[i]['ingredients'][j])
    li1 = [y for y in ingre1]
    li1 = ",".join(map(str, li1))
    full1.append(li1)
    ingre1 = []
    #if(i == 5000):
    #    break
    if(i%5000 == 0):
        print(i)
        
full2 = []
for k in range(len(full1)):
    full2.append(full1[k])

df2 = pd.DataFrame({'id':id1,'ingredients':full1})
print(df2.head())
print(df2['ingredients'][0])
ans = []

print("\n\n")
print(full2[:2])
print("\n\n\n")

for k in range(len(full1)):
    ans.append((cuis[(\
    lgr.predict(count_vect.transform([full1[k]])))]))

out = pd.DataFrame({'id':df2['id'], 'cuisine':ans})
'''df2['ingredients'] = ",".join(map(str, df2['ingredients']))
print(df2.head())
ans = pd.DataFrame(columns=['cuisine'])
k = 0
print(df2['ingredients'][0])
print("\t\t\tTesting")'''
'''for i in df2['ingredients']:
    temp = pd.DataFrame(columns=['cuisine'])
    temp['cuisine'] = (cuis[(\
    clf.predict(count_vect.transform([i])))])
    #print(i)
    print(temp)
    ans = ans.append(temp)
    k+=10
    if(k == 1):
        break'''
#print(ans)
print(out.head())
out['cuisine'] = out['cuisine'].str[0]
print(out.head())

0
5000
      id                                        ingredients
0  18009  baking powder,eggs,all-purpose flour,raisins,m...
1  28583  sugar,egg yolks,corn starch,cream of tartar,ba...
2  41580  sausage links,fennel bulb,fronds,olive oil,cub...
3  29752  meat cuts,file powder,smoked sausage,okra,shri...
4  35687  ground black pepper,salt,sausage casings,leeks...
baking powder,eggs,all-purpose flour,raisins,milk,white sugar



['baking powder,eggs,all-purpose flour,raisins,milk,white sugar', 'sugar,egg yolks,corn starch,cream of tartar,bananas,vanilla wafers,milk,vanilla extract,toasted pecans,egg whites,light rum']




          cuisine     id
0       [british]  18009
1   [southern_us]  28583
2       [italian]  41580
3  [cajun_creole]  29752
4       [italian]  35687
        cuisine     id
0       british  18009
1   southern_us  28583
2       italian  41580
3  cajun_creole  29752
4       italian  35687


In [42]:
file = "submission2.csv"
out.to_csv(file,index=False)

In [11]:
id1 = []
ingre1 = []
full1 = []
for i in range(length1):
    id1.append(test[i]['id'])
    for j in range(len
                   (test[i]['ingredients'])):
        ingre1.append(test[i]['ingredients'][j])
    li1 = [y for y in ingre1]
    li1 = ",".join(map(str, li1))
    full1.append(li1)
    ingre1 = []
    #if(i == 5000):
    #    break
    if(i%5000 == 0):
        print(i)
        
full2 = []
for k in range(len(full1)):
    full2.append(full1[k])

df2 = pd.DataFrame({'id':id1,'ingredients':full1})
print(df2.head())
print(df2['ingredients'][0])
ans = []

print("\n\n")
print(full2[:2])
print("\n\n\n")

for k in range(len(full1)):
    ans.append((cuis[(\
    lsvc.predict(count_vect.transform([full1[k]])))]))

out = pd.DataFrame({'id':df2['id'], 'cuisine':ans})
'''df2['ingredients'] = ",".join(map(str, df2['ingredients']))
print(df2.head())
ans = pd.DataFrame(columns=['cuisine'])
k = 0
print(df2['ingredients'][0])
print("\t\t\tTesting")'''
'''for i in df2['ingredients']:
    temp = pd.DataFrame(columns=['cuisine'])
    temp['cuisine'] = (cuis[(\
    clf.predict(count_vect.transform([i])))])
    #print(i)
    print(temp)
    ans = ans.append(temp)
    k+=10
    if(k == 1):
        break'''
#print(ans)
print(out.head())
out['cuisine'] = out['cuisine'].str[0]
print(out.head())

0
5000
      id                                        ingredients
0  18009  baking powder,eggs,all-purpose flour,raisins,m...
1  28583  sugar,egg yolks,corn starch,cream of tartar,ba...
2  41580  sausage links,fennel bulb,fronds,olive oil,cub...
3  29752  meat cuts,file powder,smoked sausage,okra,shri...
4  35687  ground black pepper,salt,sausage casings,leeks...
baking powder,eggs,all-purpose flour,raisins,milk,white sugar



['baking powder,eggs,all-purpose flour,raisins,milk,white sugar', 'sugar,egg yolks,corn starch,cream of tartar,bananas,vanilla wafers,milk,vanilla extract,toasted pecans,egg whites,light rum']




          cuisine     id
0       [british]  18009
1   [southern_us]  28583
2       [italian]  41580
3  [cajun_creole]  29752
4       [italian]  35687
        cuisine     id
0       british  18009
1   southern_us  28583
2       italian  41580
3  cajun_creole  29752
4       italian  35687


In [12]:
file = "submission3.csv"
out.to_csv(file,index=False)